# First part Project Week3: Website Scraping
#### Several assumptions
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

Using the package BeautifulSoup, we take the first table found on the webpage and put it in a list

In [36]:
from bs4 import BeautifulSoup
import requests

# get html source
html_source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_source, 'lxml')

# get the table from the page
table = soup.find('table')

tablelist = []
for element in table.find_all('tr'):
    tablelist.append(element.text.split('\n')[1:4])

Create dataframe and clean the data

In [54]:
# create dataframe
import pandas as pd

df = pd.DataFrame(tablelist)
df.columns = df.iloc[0]
df = df.loc[1:]

# Dropping cells with borough not assigned
na_borough = df[df['Borough'] == 'Not assigned'].index
df.drop(na_borough, axis=0, inplace=True)

# Assign neighbourhood
na_neighbourhood = df[df['Neighbourhood'] == 'Not assigned'].index
for idx in na_neighbourhood:
    df.loc[idx,'Neighbourhood'] = df.loc[idx]['Borough']
    
df = df.reset_index(drop=True)

Combine neighbourhood with the same postcode

In [55]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [56]:
df.shape

(103, 3)

# Second part Project Week3: Geocoder

Since the geocoder package goes into infinite loop, I download the csv file from the given link

In [21]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data

We can create a dataframe from the csv file

In [60]:
geospatial_data = pd.read_csv('Geospatial_data.csv')
geospatial_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)

geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now merge the geospatial_data into the neighbourhood dataframe

In [64]:
df_merge = pd.merge(df,geospatial_data,on='Postcode')
df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
